In [1]:
import importlib.util
import os
from pathlib import Path
import re
from views_pipeline_core.managers.model import ModelManager, ModelPathManager
from views_pipeline_core.managers.ensemble import  EnsembleManager, EnsemblePathManager

/home/sonja/Desktop/views-platform/views-models/envs/views_stepshifter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("..")
base_dir = os.getcwd()
target_dir = Path(base_dir+"/models/")
target_dir

PosixPath('/home/sonja/Desktop/views-platform/views-models/models')

In [10]:
target_dir

PosixPath('/home/sonja/Desktop/views-platform/views-models/models')

In [11]:
#model_manager = ModelManager(model_path=ModelPathManager(configs_dir))
model_manager = ModelManager(model_path=ModelPathManager('/home/sonja/Desktop/views-platform/views-models/models/blank_space'))

In [11]:
# Update repository structure:
def generate_repo_structure(folders, scripts, model_name):
    """Generate a structured repository tree dynamically from folders and scripts."""

    root_path = Path(folders["model_dir"])  # Root directory
    tree = [model_name]  # Start with the model name

    # Sort folders to ensure correct hierarchy
    sorted_folders = sorted(folders.values(), key=lambda x: x.count("/"))  
    folder_structure = {folder: [] for folder in sorted_folders}  

    # Assign scripts to their respective folders
    for script, script_path in scripts.items():
        parent_folder = str(Path(script_path).parent)
        if parent_folder in folder_structure:
            folder_structure[parent_folder].append(script)

    # Function to recursively build the tree
    def build_tree(current_path, depth=0):
        indent = "│   " * depth
        rel_path = Path(current_path).relative_to(root_path)
        tree.append(f"{indent}├── {rel_path.name}")

        # Add scripts in the current folder
        for script in sorted(folder_structure[current_path]):
            tree.append(f"{indent}│   ├── {script}")

        # Add subfolders in order
        subfolders = [f for f in sorted_folders if Path(f).parent == Path(current_path)]
        for subfolder in subfolders:
            build_tree(subfolder, depth + 1)


    root_scripts = []
    for key, value in scripts.items():
        #print(Path(value).parent)
        if Path(value).parent==Path(root_path):
                root_scripts.append(key)

    for script in sorted(root_scripts):
        tree.append(f"├── {script}")

    # Build tree from root
    for folder in sorted_folders:
        if Path(folder).parent == root_path:  # Start from root-level folders
            build_tree(folder)

    return "\n".join(tree)

In [154]:
repo_structure = generate_repo_structure(mpm.get_directories(), scripts, 'model_name', )
formatted_structure = f"```\n{repo_structure}\n```"
print(formatted_structure)

```
model_name
├── README.md
├── main.py
├── requirements.txt
├── run.sh
├── logs
├── artifacts
├── configs
│   ├── config_deployment.py
│   ├── config_hyperparameters.py
│   ├── config_meta.py
│   ├── config_queryset.py
│   ├── config_sweep.py
├── data
│   ├── generated
│   ├── processed
│   ├── raw
├── reports
├── notebooks
```


In [12]:
for subfolder in target_dir.iterdir():
    if subfolder.is_dir():  # Check if it's a directory
        print(f"Model: {subfolder.name}")
        #configs_dir = Path(subfolder.name+"/configs")
        configs_dir = target_dir / subfolder.name / "configs"
        model_manager = ModelManager(model_path=ModelPathManager(configs_dir))
        mpm = ModelPathManager(configs_dir)

        ## Get Meta Info
        model_name = model_manager.configs['name']
        model_name = " ".join(word.capitalize() for word in model_name.split("_"))

        algorithm = model_manager.configs['algorithm']
        if algorithm=='HurdleModel':
            classifier = model_manager.configs['model_clf']
            regressor = model_manager.configs['model_reg']
            algorithm_all = f"{algorithm} (Classifier: {classifier}, Regressor: {regressor})"
        else:
            algorithm_all = algorithm

        target = model_manager.configs['depvar']
        if isinstance(target, list):
            target = ", ".join(target)
        queryset = model_manager.configs['queryset']
        level = model_manager.configs['level']
        try:
            metrics = model_manager.configs['metrics']
        except KeyError:
            metrics = "No information provided"
        if isinstance(metrics, list):
            metrics = ", ".join(metrics)

        ## Get deployment mode 
        deployment = model_manager.configs['deployment_status']

        ## Get queryset description
        queryset_info = mpm.get_queryset()
        description = queryset_info.description
        try:
            description = " ".join(description.split())
        except AttributeError:
            description = 'No description provided'
        name = queryset_info.name

        ## Update old README file - For Bitter Symphony Model 
        scaffold_path = target_dir / "README_scaffold.md"
        readme_path = target_dir / subfolder.name / "README.md"

        # Read old README
        with open(readme_path, "r") as file:
            old_readme_content = file.read()

        # Add created sessioin if it exists

        match = re.search(r"(## Created on.*)", old_readme_content, re.DOTALL)
        if match==None:
            new_string=''
        else:
            created_section = match.group(1).strip()
            insert_position = created_section.find("##")

            # Find where the '##' ends (after '##' and the next space)
            end_of_heading = len("##")  # Skip the '##' part itself
            new_string = created_section[:end_of_heading] + " " + 'Model' + created_section[end_of_heading:]

        # Read scaffold.md content
        with open(scaffold_path, "r") as file:
            content = file.read()


        # Dictionary of placeholders and their replacements
        replacements = {
            "{{MODEL_NAME}}": model_name,
            "{{MODEL_ALGORITHM}}": algorithm_all,
            "{{LEVEL_OF_ANALYSIS}}": level,
            "{{TARGET}}": target,
            "{{FEATURES}}": name, 
            "{{DESCRIPTION}}": description,
            "{{DEPLOYMENT}}": deployment,
            "{{METRICS}}": metrics,
            "{{CREATED_SECTION}}": new_string,
        }


        # Replace placeholders in scaffold content
        for placeholder, value in replacements.items():
            content = content.replace(placeholder, value)


        repo_root = target_dir / subfolder.name
        scripts = mpm.get_scripts()
        folders = mpm.get_directories()
        scripts["run.sh"] = folders['model_dir']+'/run.sh'
        scripts["requirements.txt"] = folders['model_dir'] +'/requirements.txt'
        repo_structure = generate_repo_structure(folders, scripts, model_name=model_name)
        formatted_structure = f"```\n{repo_structure}\n```"
        formatted_structure


        updated_readme = content.replace("## Repository Structure",
                f"## Repository Structure\n\n{formatted_structure}",
            )
        
        # Write the updated content to README.md
        with open(readme_path, "w") as file:
            file.write(updated_readme)
        break

Model: chunky_cat


In [ ]:
##### Ensamble Models ###

In [157]:
folders = mpm.get_directories()
folders['model_dir']

'/home/sonja/Desktop/views-platform/views-models/models/bad_blood'

In [13]:
ens_manager = EnsembleManager(ensemble_path=EnsemblePathManager('/home/sonja/Desktop/views-platform/views-models/ensembles/cruel_summer'))
#ens_manager.configs

In [8]:
base_dir = os.getcwd()
target_ens_dir = Path(base_dir+"/ensembles/")
target_ens_dir

PosixPath('/home/sonja/Desktop/views-platform/views-models/ensembles')

In [ ]:
for subfolder in target_ens_dir.iterdir():
    if subfolder.is_dir():  # Check if it's a directory
        print(f"Model: {subfolder.name}")
        #configs_dir = Path(subfolder.name+"/configs")
        configs_dir = target_ens_dir / subfolder.name / "configs"
        ens_manager = EnsembleManager(ensemble_path=EnsemblePathManager(configs_dir))
        epm = EnsemblePathManager(configs_dir)

        ## Get Meta Info
        ens_name = ens_manager.configs['name']
        ens_name = " ".join(word.capitalize() for word in ens_name.split("_"))

        models = ens_manager.configs['models']
        models = ", ".join(models)

        target = ens_manager.configs['depvar']
        if isinstance(target, list):
            target = ", ".join(target)
        level = ens_manager.configs['level']
        try:
            metrics = ens_manager.configs['metrics']
        except KeyError:
            metrics = "No information provided"
        if isinstance(metrics, list):
            metrics = ", ".join(metrics)
        
        aggregation = ens_manager.configs['aggregation']

        ## Get deployment mode 
        deployment = ens_manager.configs['deployment_status']

        ## Update old README file - For Bitter Symphony Model 
        scaffold_path = target_ens_dir / "README_ensemble_scaffold.md"
        readme_path = target_ens_dir / subfolder.name / "README.md"

        # Read old README
        with open(readme_path, "r") as file:
            old_readme_content = file.read()

        # Add created sessioin if it exists

        match = re.search(r"(## Created on.*)", old_readme_content, re.DOTALL)
        if match==None:
            new_string=''
        else:
            created_section = match.group(1).strip()
            insert_position = created_section.find("##")

            # Find where the '##' ends (after '##' and the next space)
            end_of_heading = len("##")  # Skip the '##' part itself
            new_string = created_section[:end_of_heading] + " " + 'Model' + created_section[end_of_heading:]

        # Read scaffold.md content
        with open(scaffold_path, "r") as file:
            content = file.read()


        # Dictionary of placeholders and their replacements
        replacements = {
            "{{ENSEMBLE_NAME}}": ens_name,
            "{{MODELS}}": models,
            "{{LEVEL_OF_ANALYSIS}}": level,
            "{{TARGET}}": target,
            "{{AGGREGATION}}": aggregation,
            "{{DEPLOYMENT}}": deployment,
            "{{METRICS}}": metrics,
            "{{CREATED_SECTION}}": new_string,
        }


        # Replace placeholders in scaffold content
        for placeholder, value in replacements.items():
            content = content.replace(placeholder, value)

        scripts = epm.get_scripts()
        folders = epm.get_directories()
        scripts["run.sh"] = folders['model_dir']+'/run.sh'
        scripts["requirements.txt"] = folders['model_dir'] +'/requirements.txt'
        repo_structure = generate_repo_structure(folders, scripts, model_name=ens_name)
        formatted_structure = f"```\n{repo_structure}\n```"
        formatted_structure


        updated_readme = content.replace("## Repository Structure",
                f"## Repository Structure\n\n{formatted_structure}",
            )
        

        # Write the updated content to README.md
        with open(readme_path, "w") as file:
            file.write(updated_readme)

Model: cruel_summer
Model: pink_ponyclub
Model: white_mustang
Model: skinny_love


In [62]:
model_manager.configs

{'steps': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36],
 'parameters': {'clf': {'n_estimators': 200}, 'reg': {'n_estimators': 200}},
 'name': 'old_money',
 'algorithm': 'HurdleModel',
 'model_clf': 'LGBMClassifier',
 'model_reg': 'LGBMRegressor',
 'metrics': ['RMSLE', 'CRPS'],
 'depvar': 'ln_ged_sb_dep',
 'queryset': 'fatalities003_pgm_escwa_drought',
 'level': 'pgm',
 'creator': 'Xiaolong',
 'deployment_status': 'shadow'}

In [7]:
mpm = ModelPathManager("/home/sonja/Desktop/views-platform/views-models/models/bad_blood/logs")

In [63]:
mpm.get_directories()

{'model_dir': '/home/sonja/Desktop/views-platform/views-models/models/old_money',
 'logging': '/home/sonja/Desktop/views-platform/views-models/models/old_money/logs',
 'artifacts': '/home/sonja/Desktop/views-platform/views-models/models/old_money/artifacts',
 'configs': '/home/sonja/Desktop/views-platform/views-models/models/old_money/configs',
 'data': '/home/sonja/Desktop/views-platform/views-models/models/old_money/data',
 'data_generated': '/home/sonja/Desktop/views-platform/views-models/models/old_money/data/generated',
 'data_processed': '/home/sonja/Desktop/views-platform/views-models/models/old_money/data/processed',
 'reports': '/home/sonja/Desktop/views-platform/views-models/models/old_money/reports',
 'data_raw': '/home/sonja/Desktop/views-platform/views-models/models/old_money/data/raw',
 'notebooks': '/home/sonja/Desktop/views-platform/views-models/models/old_money/notebooks'}

In [65]:
mpm.view_directories()


Name                	Path                                              
root                	/home/sonja/Desktop/views-platform/views-models   
logging             	/home/sonja/Desktop/views-platform/views-models/models/old_money/logs
artifacts           	/home/sonja/Desktop/views-platform/views-models/models/old_money/artifacts
configs             	/home/sonja/Desktop/views-platform/views-models/models/old_money/configs
data                	/home/sonja/Desktop/views-platform/views-models/models/old_money/data
data_generated      	/home/sonja/Desktop/views-platform/views-models/models/old_money/data/generated
data_processed      	/home/sonja/Desktop/views-platform/views-models/models/old_money/data/processed
reports             	/home/sonja/Desktop/views-platform/views-models/models/old_money/reports
data_raw            	/home/sonja/Desktop/views-platform/views-models/models/old_money/data/raw
notebooks           	/home/sonja/Desktop/views-platform/views-models/models/old_money/notebo

In [16]:
pretend = ModelPathManager("orange_cat", validate=False)

In [17]:
pretend.get_directories()

{'model_dir': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat',
 'logging': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/logs',
 'artifacts': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/artifacts',
 'configs': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/configs',
 'data': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/data',
 'data_generated': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/data/generated',
 'data_processed': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/data/processed',
 'reports': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/reports',
 'data_raw': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/data/raw',
 'notebooks': '/home/sonja/Desktop/views-platform/views-models/models/orange_cat/notebooks'}

In [19]:
mpm.models

PosixPath('/home/sonja/Desktop/views-platform/views-models/models')

In [20]:
mpm.root

PosixPath('/home/sonja/Desktop/views-platform/views-models')

In [22]:
mpm.get_latest_model_artifact_path(run_type="forecasting")

FileNotFoundError: No model artifacts found for run type 'forecasting' in path '/home/sonja/Desktop/views-platform/views-models/models/bad_blood/artifacts'

In [23]:
mpm.get_queryset()

Queryset(name='fatalities003_pgm_natsoc', loa='priogrid_month', themes=['fatalities'], description='Fatalities natural and social geography, pgm level\n\n                                    Predicting ln(fatalities) using natural and social geography features\n\n                                    ', operations=[[RenameOperation(namespace='trf', name='util.rename', arguments=['ln_ged_sb_dep']), TransformOperation(namespace='trf', name='ops.ln', arguments=[]), TransformOperation(namespace='trf', name='missing.replace_na', arguments=[]), DatabaseOperation(namespace='base', name='priogrid_month.ged_sb_best_sum_nokgi', arguments=['values'])], [RenameOperation(namespace='trf', name='util.rename', arguments=['ln_ged_sb']), TransformOperation(namespace='trf', name='missing.fill', arguments=[]), TransformOperation(namespace='trf', name='ops.ln', arguments=[]), DatabaseOperation(namespace='base', name='priogrid_month.ged_sb_best_sum_nokgi', arguments=['values'])], [RenameOperation(namespace='tr